In [41]:
import qiskit as qk
import numpy as np
import scipy.linalg as lin
import math
from qiskit.quantum_info import Statevector , Operator , partial_trace , DensityMatrix
from qiskit.circuit.library.standard_gates import HGate , XGate
from qiskit.circuit.library import GlobalPhaseGate
from qiskit.extensions import Initialize , UnitaryGate
from qiskit.providers.aer import AerSimulator
import sys
import matplotlib.pyplot as plt
from qiskit_ibm_provider import IBMProvider
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import BackendEstimator
from qiskit import QuantumCircuit, Aer, transpile
import importlib
import PMR_LCU
importlib.reload(PMR_LCU)  # Reload to reflect updates
from PMR_LCU import *

/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/Quantum_Circuit_Github/Quantum_Circuits_Floquet/PMR_LCU.py:97: SyntaxWarning: invalid escape sequence '\o'
  """
/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/Quantum_Circuit_Github/Quantum_Circuits_Floquet/PMR_LCU.py:263: SyntaxWarning: invalid escape sequence '\o'
  """"
/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/Quantum_Circuit_Github/Quantum_Circuits_Floquet/PMR_LCU.py:298: SyntaxWarning: invalid escape sequence '\o'
  """"
/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/Quantum_Circuit_Github/Quantum_Circuits_Floquet/PMR_LCU.py:337: SyntaxWarning: invalid escape sequence '\o'
  """


## Parameters of the system

Here, we define the parameters for the simulation of the Hamiltonian $H(t) = \sum_{i=1}^n h_i Z_i + V_x \sum_{i=1}^{n-1} Z_i Z_{i+1} + cos(\omega t) \sum_{i=1}^{n-1} c_i X_i X_{i+1}$.  

In [42]:
#====================== Circuit Paramters ================
number_of_spins = 4 # number of particles (qubits)
K = 2     # only two modes -w and w
M = number_of_spins-1  # number of permutation operators

C0 = 1.0 # Parameter for the floquet interaction strength
Coeffs = C0*np.array([2+(-1)**x for x in np.arange(1,number_of_spins)]) # there are n elements in this array
Omega = 1.0
h0 = 1.0
Longit_h = h0*np.array([1.5 + 0.5*(-1)**x for x in range(number_of_spins)])
Vx = [1.0]*M
C = np.max(Coeffs)

Gammas_i = Coeffs
Gammas_k = [0.5]*K
Qmax = 2

Gamma_1 = np.sum(Gammas_i)
Gamma_2 = np.sum(Gammas_k)
Gamma = Gamma_2 * Gamma_1
Gamma_list = [Gammas_i , Gammas_k]
Delta_t = np.log(2)/Gamma
GDt = Gamma*Delta_t

NumberOfTimeSteps = 2
FinalTime = NumberOfTimeSteps*Delta_t

print(f'The simulation parameters are ... ')
print(f'V={Vx} hs = {Longit_h} Gamma_list = {Gamma_list} time = {FinalTime} Omega = {Omega} Number of time steps = {NumberOfTimeSteps}')

The simulation parameters are ... 
V=[1.0, 1.0, 1.0] hs = [2. 1. 2. 1.] Gamma_list = [array([1., 3., 1.]), [0.5, 0.5]] time = 0.2772588722239781 Omega = 1.0 Number of time steps = 2


### Reading in exact numerical results

In [15]:
InitialStateString = '0'*number_of_spins
# Read in the theoretical (exact numerical) values of the final state, given the initial state!
import json 

with open("./Exact_Results/FinalState_params1_numoftimestep_"+str(NumberOfTimeSteps)"_init_"+InitialStateString+".json", "r") as f:
    data = json.load(f)

# Extract the first term (complex number) from each entry
complex_numbers = []
for entry in data:
    # Extract everything before the first comma (first term)
    first_term = entry.split(",")[0].strip(" {}")  # Remove spaces and braces
    # Replace " I" with "j" (Python complex notation)
    first_term = first_term.replace(" ", "").replace("I", "j")
    # Append to list
    complex_numbers.append(first_term)

# Convert strings to complex numbers
FinalStateExact = np.array([complex(num) for num in complex_numbers], dtype=np.complex128)

print(f'The exact vector is {FinalStateExact}')

The exact vector is [-0.359907-0.27317j    0.      +0.j         0.      +0.j
 -0.240997-0.0828557j  0.      +0.j         0.235109-0.260398j
  0.565744-0.0728501j  0.      +0.j         0.      +0.j
 -0.207041+0.0646636j -0.223505-0.364826j   0.      +0.j
 -0.121577+0.158671j   0.      +0.j         0.      +0.j
  0.109746-0.0185018j]


### Initializing the Quantum circuit

In [43]:
Nkq = int(np.log2(K))*Qmax
Niq = int(np.log2(number_of_spins))*Qmax
Ntotal = Nkq + Niq + number_of_spins + Qmax + 1

kqQubits = qk.QuantumRegister(Nkq , '|kq>')
iqQubits = qk.QuantumRegister(Niq , '|iq>')
zQubits = qk.QuantumRegister(number_of_spins , '|z>')
qQubits = qk.QuantumRegister(Qmax , '|q>')
ancQubit = qk.QuantumRegister(1 , '|anc>')

kq_qbits_index = 0
iq_qbits_index = 1
z_qbits_index = 2
q_qbits_index = 3
anc_qbits_index = 4

# =========================== Building the circuits  U_0(dt) U_od(dt) ... U_0(dt) U_od(dt) =================================== #

FullCirc = qk.QuantumCircuit( kqQubits , iqQubits , zQubits , qQubits , ancQubit )
FullCircwithPS = qk.QuantumCircuit( kqQubits , iqQubits , zQubits , qQubits , ancQubit )


numberofsteps = FinalTime
Prepare_full_unitary( FullCirc , [Longit_h , Vx] , Omega , Delta_t , Gamma_list , [kq_qbits_index , iq_qbits_index , z_qbits_index , q_qbits_index , anc_qbits_index] , NumberOfTimeSteps )
Prepare_full_unitary_wo_Rgate( FullCircwithPS , [Longit_h , Vx] , Omega , Delta_t , Gamma_list , [kq_qbits_index , iq_qbits_index , z_qbits_index , q_qbits_index , anc_qbits_index] , NumberOfTimeSteps )

#print(FullCirc.draw())
#Ops = SparsePauliOp.from_list([('I'*(Nkq + Niq)+'XIII'+'I'*Q , 1) , ('I'*(Nkq + Niq)+'IXII'+'I'*Q , 1) , ('I'*(Nkq + Niq)+'ZIII'+'I'*Q , 1) , ('I'*(Nkq + Niq)+'IZII'+'I'*Q , 1)])
#print(f'Circuit Prepared! The parameters are:  Vx = {Vx} , hs = {Longit_h} , Omega = {Omega} , C0 = {C0} , Q = {Q} , Gamma = {Gam} , Delta_t = {Delta_t} , GDt = {Gam*Delta_t}') 

### State initialization and simulation

In [50]:
# Initializing the state:
InitStateZ = Statevector.from_label('0'*number_of_spins)

InitState = Statevector.from_label('0'*Nkq)
InitState = Statevector.from_label('0'*Niq).tensor(InitState)
InitState = InitStateZ.tensor(InitState)
InitState = Statevector.from_label('0'*Qmax).tensor(InitState)
InitState = Statevector.from_label('0').tensor(InitState)


FinalStateFull = InitState.evolve(FullCirc)
FinalStatewoRgate = InitState.evolve(FullCircwithPS)

# Post-selection on the FinalStatewoRgate:
increment = 2**( (Niq + Nkq) )
HighestMultiple = int( 2**( number_of_spins ) )
print(f'The highest multiple is {HighestMultiple}')
FinalStatePS = np.array( [FinalStatewoRgate[i*increment] for i in range(HighestMultiple)] )
FinalStatePS = FinalStatePS/np.linalg.norm( FinalStatePS )

FinalStatewithR = np.array( [FinalStateFull[i*increment] for i in range(HighestMultiple)] )
FinalStatewithR = FinalStatewithR/np.linalg.norm( FinalStatewithR )


def state_overlap( state1 , state2):
    l1 = len(state1)
    l2 = len(state2)
    print(f'l1 is {l1} and l2 is {l2}')
    if l1 != l2:
        ValueError("The dimensions of the states don't match!")
    else:
        return np.dot(np.conjugate(state1) , state2)

print(f'The overlap of the Post selected with the Rgate final state is {state_overlap(FinalStateFull , FinalStatePS)}')

The highest multiple is 128
l1 is 8192 and l2 is 128
The overlap of the Post selected with the Rgate final state is None


In [36]:
a = Statevector.from_label('01')
b = a.tensor(Statevector.from_label('1'))
testC = QuantumCircuit(3)
testC.x(0)
print(testC.draw())
print(b)
print(b.evolve(testC))

     ┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
          
q_2: ─────
          
Statevector([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j],
            dims=(2, 2, 2))
Statevector([0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j],
            dims=(2, 2, 2))


In [162]:
# Transpiling the circuit and getting a gate count:
backend_sim = Aer.get_backend('qasm_simulator')
transpiled_circuit = transpile(FullCirc, backend_sim)

gate_count = transpiled_circuit.count_ops()
gate_depth = transpiled_circuit.depth()
Toffoli_count = gate_count.get('ccx' , 0)
CNOT_count = gate_count.get('cx' , 0)
#print("\nGate Count:", gate_count)
print(f'The gate depth is {gate_depth} , total number of qubits required is {Ntotal}, the number of Toffoli gates is {Toffoli_count}, and number of CNOTs is {CNOT_count}')

#print(FullCirc.draw())

The gate depth is 308 , total number of qubits required is 10, the number of Toffoli gates is 36, and number of CNOTs is 158


In [163]:
IBMProvider.save_account(token='1a6f8188ff1654f78d9c4418c7382addf037bf1aca1ec8a40fe866c40123a30913609310721808e16cf25f3f5f92de14f163d30a8162b5a0bba7bda1324ca1cc', overwrite=True )
provider = IBMProvider(instance="usc/hen-research-g/hen-lab")

backend = provider.get_backend('ibm_strasbourg')
Estimator = BackendEstimator(backend)
job = Estimator.run( [FullCirc]*len(Ops) , [Ops[i] for i in range(len(Ops))]  , shots = 5000 )
id=job.job_id()
print(f'The job id is {id}')

The job id is 3978659f-249a-4f61-bd27-78e21a3ca017


In [164]:
results = job.result()
#counts = results.get_counts()


In [165]:
values = results.values
metadata = results.metadata
print(f'The values are {values} and the metadata is {metadata}')

The values are [ 0.0176  0.138   0.942  -0.1512] and the metadata is [{'variance': 0.99969024, 'shots': 5000}, {'variance': 0.980956, 'shots': 5000}, {'variance': 0.11263600000000007, 'shots': 5000}, {'variance': 0.97713856, 'shots': 5000}]
